<a href="https://colab.research.google.com/github/nikitamahajan92/ChatBot/blob/main/Chatbot_using_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Import the libraries:**

In [16]:
import tensorflow
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout , Activation, Flatten , Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD
import random
import json
import pickle

In [17]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [18]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# **2. Declaring Constants:**

In [19]:
words=[]
labels = []
docs = []
ignore_list = ['?', '!']

## **3. Loading our dataset that is intents.json file:**

In [22]:
dataset = open('intents.json').read()
intents = json.loads(dataset)

# **4. Preprocess Data:**

In [23]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        word_token = nltk.word_tokenize(pattern)
        words.extend(word_token)
        #add documents in the corpus
        docs.append((word_token, intent['tag']))
        # add to our labels list
        if intent['tag'] not in labels:
            labels.append(intent['tag'])

# **5. Lemmatizing Each word:**

In [24]:
# lemmatize each word, and sort words by removing duplicates:
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_list]
words = sorted(list(set(words)))
# sort labels:
labels = sorted(list(set(labels)))

# **6. Save words and labels list (using pickle):**

In [25]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(labels,open('labels.pkl','wb'))

# **7. Creating our Training data:**

In [26]:
# creating our training data:
training_data = []
# creating an empty array for our output (with size same as length of labels):
output = [0]*len(labels)
for doc in docs:
    bag_of_words = []
    pattern_words = doc[0]
    #lemmatize pattern words:
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    for w in words:
        if w in pattern_words:
            bag_of_words.append(1)
        else:
            bag_of_words.append(0)
            
    output_row = list(output)
    output_row[labels.index(doc[1])] = 1
    
    training_data.append([bag_of_words,output_row])

# **8. Shuffle and Convert our Training data to array:**

In [27]:
# convert training_data to numpy array and shuffle the data:
random.shuffle(training_data)
training_data = np.array(training_data)

<ipython-input-27-d2a7c1a73fe0>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training_data = np.array(training_data)


# **9. Splitting the data into x_train and y_train:**

In [28]:
# Now we have to create training list:
x_train = list(training_data[:,0])
y_train = list(training_data[:,1])

# **10. Model Creation:**

In [29]:
# Creating Model:
model = Sequential()
model.add(Dense(128, input_shape=(len(x_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation='softmax'))

# **11. Model Summary:**

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               11392     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 9)                 585       
                                                                 
Total params: 20,233
Trainable params: 20,233
Non-trainable params: 0
_________________________________________________________________


# **12. Compile and Fit our model to find the accuracy:**

In [31]:
from tensorflow.keras.optimizers import legacy

sgd_optimizer = legacy.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd_optimizer, metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [32]:
# fit the model 
history = model.fit(np.array(x_train), np.array(y_train), epochs=200, batch_size=5, verbose=1)


Epoch 1/200
10/10 [==============================] - 4s 6ms/step - loss: 2.2542 - accuracy: 0.1489 
Epoch 2/200
10/10 [==============================] - 0s 6ms/step - loss: 2.1694 - accuracy: 0.1915
Epoch 3/200
10/10 [==============================] - 0s 4ms/step - loss: 2.0791 - accuracy: 0.2128
Epoch 4/200
10/10 [==============================] - 0s 5ms/step - loss: 1.9275 - accuracy: 0.3404
Epoch 5/200
10/10 [==============================] - 0s 5ms/step - loss: 1.8530 - accuracy: 0.4468
Epoch 6/200
10/10 [==============================] - 0s 5ms/step - loss: 1.6979 - accuracy: 0.4255
Epoch 7/200
10/10 [==============================] - 0s 5ms/step - loss: 1.6955 - accuracy: 0.3830
Epoch 8/200
10/10 [==============================] - 0s 5ms/step - loss: 1.5566 - accuracy: 0.4468
Epoch 9/200
10/10 [==============================] - 0s 5ms/step - loss: 1.2917 - accuracy: 0.6383
Epoch 10/200
10/10 [==============================] - 0s 5ms/step - loss: 1.1183 - accuracy: 0.6383
Epoch 11

# **13. Save the model:**

In [33]:
model.save('chatbot_Application_model.h5', history)

# **14. Final step to predict the sentences and get responses:**

In [34]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
import json
import random
from keras.models import load_model
model = load_model('chatbot_Application_model.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
labels = pickle.load(open('labels.pkl','rb'))

In [35]:
def bank_of_words(s,words, show_details=True):
    bag_of_words = [0 for _ in range(len(words))]
    sent_words = nltk.word_tokenize(s)
    sent_words = [lemmatizer.lemmatize(word.lower()) for word in sent_words]
    for sent in sent_words:
        for i,w in enumerate(words):
            if w == sent:
                bag_of_words[i] = 1
    return np.array(bag_of_words)
def predict_label(s, model):
    # filtering out predictions
    pred = bank_of_words(s, words,show_details=False)
    response = model.predict(np.array([pred]))[0]
    ERROR_THRESHOLD = 0.25
    final_results = [[i,r] for i,r in enumerate(response) if r>ERROR_THRESHOLD]
    final_results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in final_results:
        return_list.append({"intent": labels[r[0]], "probability": str(r[1])})
    return return_list

In [36]:
def Response(ints, intents_json):
    tags = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tags):
            response = random.choice(i['responses'])
            break
    return response
def chatbot_response(msg):
    ints = predict_label(msg, model)
    response = Response(ints, intents)
    return response

In [ ]:
def chat():
    print("Start chat with ChatBot of Medical_Help")
    while True:
        inp = input("You: ")1
        
        if inp.lower() == 'quit':
            break
        response = chatbot_response(inp)
        print("\n BOT: " + response + '\n\n')
chat()

Start chat with ChatBot of Medical_Help
You: Hello
1/1 [==============================] - 0s 28ms/step

 BOT: Hi there, how can I help?


You: How you could help me?
1/1 [==============================] - 0s 20ms/step

 BOT: Offering support for Adverse drug reaction, Blood pressure, Hospitals and Pharmacies


You: How to check Adverse drug reaction?
1/1 [==============================] - 0s 19ms/step

 BOT: Navigating to Adverse drug reaction module


You: I want to log blood pressure results
1/1 [==============================] - 0s 31ms/step

 BOT: Navigating to Blood Pressure module


You: Blood pressure for patient
1/1 [==============================] - 0s 21ms/step

 BOT: Please provide Patient ID


You: 236261
1/1 [==============================] - 0s 21ms/step

 BOT: Hello, thanks for asking


You: poponnklnkb
1/1 [==============================] - 0s 29ms/step

 BOT: Hi there, how can I help?


You: 
1/1 [==============================] - 0s 19ms/step

 BOT: Good to see you ag